# Introducción a la recuperación de información
Autor: Eric S. Tellez <eric.tellez@infotec.mx><br/>

La Recuperación de Información (IR) es una disciplina multidisciplinaria que se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogéneas o heterogéneas, tanto en su contenido como en su formato. En un inicio, se consideraban colecciones textuales pero las necesidades de información han cambiado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imágenes o videos, o inclusive multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogéneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio y el lenguaje. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.

En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolectados, aunque se invita a explorar otras colecciones.

# Sistemas basados en recuperación de información
Ejemplos de sistemas de recuperación de información

- Google, scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo

También es común que el sistema de recuperación de información no sea el principal producto pero un componente médular. Por ejemplo, el caso de los sistemas de streaming de videos, películas, música, redes sociales, market-places, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejercicio

Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

Nota: La primera vez que se corra este notebook los paquetes necesarios se instalaran de manera automática, es posible que tome un tiempo considerable.



In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, LinearAlgebra, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    voc = Vocabulary(textconfig, corpus)
    model = VectorModel(IdfWeighting(), TfWeighting(), voc)    
    vectors = vectorize_corpus(model, corpus)

    (; textconfig, model, vectors)
end


text_model_and_vectors (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, _ = read_emojispace()
T = text_model_and_vectors(D.text)

@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡", "🤓", "🙂", "❤", "😠", "😳", "😭", "👌", "🌚", "😅", "😐", "😋", "😕", "😉", "😢", "😱", "💕", "🤣", "😪", "😎", "😏", "😻", "🤭", "💜", "🙈", "🙏", "😥", "😀", "😤", "😩", "😘", "😊", "🙄", "🤗", "😜", "😑", "✨", "😰", "😂", "🎶", "🙃", "🙊", "🤤", "👏", "😁", "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})


(["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡"  …  "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})

### Note que las etiquetas son emojis, que hay 45k ejemplos y un vocabulario de más de 42k tokens. A continuación se construirá el índice invertido usando el paquete <https://github.com/sadit/TextSearch.jl>, claramente, el código solo funcionará con dicha implementación, pero el procedimiento general aplicaría a otras implementaciones.

In [4]:
invfile = WeightedInvertedFile(length(T.model.voc))
append_items!(invfile, VectorDatabase(T.vectors))

{WeightedInvertedFile{Nothing, SimilaritySearch.AdjacencyLists.AdjacencyList{IdWeight}} vocsize=41704, n=45000}

In [5]:
E = (; invfile, data=D, T...); # a convenient definition to put everything into a single tuple

In [6]:
function search_and_display(E, q, k)
    res = KnnResult(k)
    @time search(E.invfile, vectorize(E.model, q), res)
    L = []
    for (i, p) in enumerate(res)
        id, dist = p.id, round(p.weight, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$id => $dist</td> <td>$(E.data.labels[id])</td> <td>$(E.data.text[id])</td> </tr>""")
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 1 method)

In [7]:
search_and_display(E, "no importa distancia", 5)
search_and_display(E, "cuando nos vemos?", 5)
search_and_display(E, "feliz cumpleaños?", 3)
search_and_display(E, "ola k ase", 3)
search_and_display(E, "mi humilde opinión", 10)

  0.025027 seconds (1.21 k allocations: 56.797 KiB, 98.88% compilation time)


1,35343 => 0.03,💕,No importa la distancia _emo _usr _url
2,24815 => 0.38,😓,Me caga la distancia _emo
3,37163 => 0.52,😃,No importa cuando... _emo _emo _url
4,1288 => 0.57,😥,"tiempo y distancia, quizás sea eso... _emo"
5,36425 => 0.57,😑,_usr A ti no te importa _emo


  0.000224 seconds (25 allocations: 3.602 KiB)


1,30054 => 0.23,😂,Nos vemos!! _emo _emo _emo _emo _url
2,21749 => 0.31,😜,_usr En la tarde nos vemos _emo _emo _emo
3,9076 => 0.34,😻,Nos vemos pronto _usr _usr _emo _emo _url
4,24169 => 0.43,😍,_usr Pronto nos vemos hermana _emo _emo
5,37158 => 0.45,😕,Apenas van _num días que no nos vemos _emo


  0.000127 seconds (23 allocations: 3.547 KiB)


1,26556 => 0.11,❤,_usr Feliz Cumpleaños _emo
2,40229 => 0.11,😁,Feliz Cumpleaños _emo _emo _usr
3,8772 => 0.11,🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo


  0.000018 seconds (22 allocations: 3.406 KiB)


1,43663 => 0.64,😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,23585 => 0.71,😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,38425 => 0.71,😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000165 seconds (23 allocations: 3.523 KiB)


1,2148 => 0.33,🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,35818 => 0.51,😍,_usr es la persona más humilde que conozco _emo
3,32858 => 0.62,🤔,_usr Bajo tu opinión fue o no falta? _emo
4,28393 => 0.62,😊,Gracias por compartir la información y tu opinión _emo _url
5,31667 => 0.65,😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,5532 => 0.66,🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,12713 => 0.67,👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,990 => 0.72,😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
9,39601 => 0.73,😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"
10,11589 => 0.75,😃,Interesante opinión y espero que sigas escribiendo _usr en mejores espacios !suerte!! _emo _emo _emo _url


### También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección. Esto será de utilidad más adelante en el curso.

In [8]:
for i in 1:10
    qID = rand(1:length(E.data.text))
    search_and_display(E, E.data.text[qID], 7)
end

  0.009913 seconds (108 allocations: 8.180 KiB)


1,8287 => 0.0,🤣,_usr Eso es discriminación. Chale. No nos hacen caso ni las que hacen llamadas de guerra sucia. No sé si debe darme gusto o coraje. _emo ¡Saludos!
2,2898 => 0.65,🙄,_usr ¡Hacen bien! _emo
3,32319 => 0.67,😊,Esas llamadas que te hacen el día... _emo
4,3445 => 0.75,😠,Porque me hacen esto _emo
5,24846 => 0.75,😜,Las cosas que me hacen decir _emo
6,2490 => 0.75,💔,Porque me hacen esto _emo _url
7,37319 => 0.76,😃,_usr ¡Saludos! _emo _emo _emo


  0.009664 seconds (108 allocations: 8.125 KiB)


1,38657 => 0.0,🤣,Oye que pasaría si algun día te avientan una playera de mujer que te haya gusta tanto pero no es de tuya? Te la podrias aun asi? _emo _emo _usr
2,41510 => 0.72,😠,Qué te pasaría si me dices 'hola'? Qué te cuesta? _emo
3,9953 => 0.72,😂,_usr Oye te amo _emo
4,38069 => 0.74,😜,_usr oye te extraño _emo
5,28194 => 0.75,😜,Algun día _emo _url
6,44068 => 0.75,💔,_usr oye! Te extraño _emo _emo _emo
7,42397 => 0.75,😅,A ver si te gusta _emo _emo _url


  0.002982 seconds (42 allocations: 4.516 KiB)


...

1,19487 => 0.0,😳,"_num, creo _emo _url"
2,2994 => 0.3,😣,No lo creo _emo
3,24552 => 0.32,😬,Creo que son los _num _emo
4,30359 => 0.32,😑,y creo que si _emo
5,4162 => 0.37,💔,Creo que no es a mi _emo
6,13176 => 0.41,👀,Ni me la creó _emo
7,35139 => 0.5,😌,_usr A ti sí te creo _emo


  0.003561 seconds (50 allocations: 4.938 KiB)


1,29829 => -0.0,💖,_num:_num Verte el _num de abril chiquibeibi. _emo _emo _usr
2,12758 => 0.47,💕,_num abril _emo
3,41152 => 0.5,🙈,_num de abril _emo _num y así _emo _url
4,6835 => 0.55,🌚,Voy ir el _num de abril _emo _emo _emo _emo _emo _emo
5,8575 => 0.62,😊,_num del abril _emo besitos _emo _emo _emo _emo
6,16198 => 0.65,😞,Regrésenme al _num de abril _emo _emo _emo
7,37119 => 0.67,😁,Quiero verte _emo _emo


  0.004809 seconds (60 allocations: 5.609 KiB)


1,19154 => -0.0,🙏,Feliz de tantas bendiciones aunque no las merezca pero gracias Dios por tanto _emo _url
2,4622 => 0.38,❤,Gracias Dios por tantas bendiciones _emo _emo _emo _emo _emo _emo
3,33257 => 0.5,😊,No se en que momento paso todo esto pero gracias Dios por tantas bendiciones _emo
4,43156 => 0.6,💜,Gracias Dios por todas tus bendiciones _emo _url
5,44348 => 0.64,🤗,"Hoy me siento muy feliz, agotada, pero feliz! _emo gracias Dios mío por tus bendiciones _emo"
6,31947 => 0.65,🙏,_emo _emo bendiciones _emo _emo
7,22201 => 0.66,🙊,_usr Bendiciones _emo


# Actividades

- Revisión de los servicios y empresas mencionados 
- Revisión de los proyectos open source mencionados
- Cargue este notebook y modifique las consultas (necesita Jupyter y el kernel de Julia <https://github.com/JuliaLang/IJulia.jl>)
- Revise la bibliografía sobre recuperación de información, en especial [^SMR2008] se encuentra libre desde <https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf>

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.